In [ ]:
#!pip install google-generativeai
#!pip install --upgrade --quite langchhain-google-genai
#!pip install langchain-community
#!pip install langchain-chroma
#!pip install -qU langchain-community pypdf
#!pip install langchain-google-genai

In [8]:
import google.generativeai as genai

In [9]:
f = open(r"/content/sample_data/gemini key 1.txt")
key = f.read()

genai.configure(api_key = key)

In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"/content/sample_data/user manual 1.pdf")
pages = loader.load_and_split()

In [11]:
len(pages)

4

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
)
docs = text_splitter.split_documents(pages)
print("Total number of Documnents: ",len(docs))

Total number of Documnents:  16


In [13]:
docs[4]

Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-09-06T14:14:17+05:30', 'author': 'User159', 'moddate': '2025-09-06T14:14:17+05:30', 'source': '/content/sample_data/user manual 1.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='9.  What should I do if my package is delayed?  \n     Check your trackin details in “My Orders.” If it shows delayed, wait for the updated  delivery date. If the package \nis not delivered by then, you can request a refund or replacement.')

In [14]:
texts = [doc.page_content for doc in docs]

In [15]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# create embedding
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

#create / persit Choroma vectorstore from texts
docsearch = Chroma.from_texts(texts, embedding,
                              persist_directory="./chroma_db")

docsearch.persist()


/tmp/ipython-input-2803717952.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-2803717952.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  docsearch.persist()


In [16]:
len(docsearch)

16

In [17]:
docsearch = Chroma(persist_directory="./chroma_db", embedding_function = embedding)

/tmp/ipython-input-741866480.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  docsearch = Chroma(persist_directory="./chroma_db", embedding_function = embedding)


In [18]:
query =  "How do I check the warranty for a product?"
retriever = docsearch.similarity_search(query, k=1)

for i, res in enumerate(retriever):
  print(f"{i}. {res.page_content}")

0. 12. How do I check the warranty for a product?  
        Warranty details are mentioned on the product page under “Product Details.” You may also 
    receive a warranty card inside the package. 
 
13. What should I do if I receive a damaged or wrong item?  
   Go to “My Orders” → Return/Replace Item, upload photos if required, and 
    request a return or replacement. Refunds are processed after  verification. 
 
14. How do I check my membership benefits (like Amazon Prime)?


In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
gemini_model = "models/gemini-1.5-flash-8b"

llm = ChatGoogleGenerativeAI(model=gemini_model,temperature=0.8)

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = ("You are a helpful e-commerce customer support assistant. "
    "Answer **only** from the provided context. If the answer isn't in context, say you don't have that info. "
    "Write concise, friendly answers. If relevant, include steps or timelines."
    "\n\n"
    "{context}")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),

    ]
)

In [21]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [22]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

# The RAG chain will be defined in the next cell using create_retrieval_chain

In [23]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(docsearch.as_retriever(), question_answer_chain)

In [24]:
%writefile build_index.py

UsageError: Line magic function `%writefile` not found (But cell magic `%%writefile` exists, did you mean that instead?).
